#Intelligent Systems - Assignment 4 - Machine Learning

- Member Names: Gorka Legarreta, Guillermo Pérez
- Group ID: B 
- Assignment: 4
- Date:6/5/2022
- Time spent in the assignment: 8 hours
- Proportion of effort done by members of the group: 50% and 50%
- Doubts and difficulties that arose during the realization: The value with the Electric fuel type give us problems with the algorithm, as there was only one row with that value. 


---

**General instructions**
 1. If you use a function of a certain library, do not forget to include the corresponding command to install or load the library
 2. Do not forget to comment on the code, especially those non-trivial commands (part of the rating depends on the cleaning of the code)
 3. It is strongly recommended to test any implemented function in order to    check for its proper operation

---

 **Submission**
- Print the notebook in a PDF file with all the sections expanded and all the cells executed, in a way all the code can be seen, and all the results are shown.
- Save the notebook as ipynb file.
- Submit both files, in addition to any input file needed for its execution in a zip folder into ALUD

---

**Evaluation**
- Total Evaluation - 25%
  - Correct and error-free execution
  - Efficiency and cleanness of the code
  - Documentation and analysis of results

---
**Deadline**: 6th May 2022

Imports of the libraries that we need:

In [ ]:
# About 4 minutes
# Clear Environment
rm(list=ls())

# Clear plots & console
if(!is.null(dev.list())) dev.off()
cat("\014") 

# Install required packages

install.packages('caret')
library(caret)


In [ ]:
library(ggplot2) ###
library(lattice) ###
library(glue) 
library(rpart)
install.packages('rpart.plot')
library(rpart.plot) 

## Linear Regression




### Data Description 📚

*Medical Cost Personal Datasets* is a dataset containing registers about the cost of an insurance joint with information regarding the age, sex, bmi, number of children...

It can be [downloaded here](https://github.com/stedy/Machine-Learning-with-R-datasets/blob/master/insurance.csv) (but you have a copy in the Drive - "insurance").

Columns in the data are:
- age: age of primary beneficiary
- sex: insurance contractor gender, female, male
- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
- children: Number of children covered by health insurance / Number of dependents
- smoker: Smoking
- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
- charges: Individual medical costs billed by health insurance



###  You are asked to: 🚨

1. Read the provided file and store in a data.frame
2. Repeat 10 times:
  - Randomly divide data in train (80%) and test (20%)
    - 🔎 For this, you can use the [createDataPartition](https://www.rdocumentation.org/packages/caret/versions/6.0-86/topics/createDataPartition) function of the *caret* package as:
    - createDataPartition(y = **data$Target**, p = **percentage**, list = FALSE)
  - Train a linear regression model
    - 🔎 For this, you have to use the [lm](https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/lm) function as:
    - model = lm(**Target~xxxxxxx**, **training.data**)
  - Make predictions over the test data
    - 🔎 You will have to use the predict function as:
    - predictions = predict(**model**,**test.data**)
  - Calculate and print Mean Absolute Error (MAE) and (MSE) obtained in the test data.

3. Print a table with the iteration (1:10), MAE and MSE.

4. Do the needed operations to answer following questions:
  - On average, if a person quits smoking, how much will the charges will be reduced?
  - Same if a non-smoker starts smoking... how much will the charges be increased?
  - Which are the 3 persons whos charge will be increased most from now to 5 years?

**You have to present:**
- Table with MAE & MSE results of the 10 iterations
- Average MAE & MSE and Standar Deviation get from the 10 iterations 
- Answer to the provided questions.

### Your solution here 💻

Read the CSV file and create the data frame:

In [ ]:
data = read.csv('insurance.csv')

Create data partitions of data, 80% training and 20% test data:

In [ ]:
training_index = createDataPartition(y = data$charges, p = 0.8, list = FALSE)

Split training data and test data:

In [ ]:
training_data = data[training_index,]
test_data = data[-training_index,]

###Train 10 times the model and select indeed, the model with best results:

In [ ]:
#We read the data
data = read.csv('insurance.csv')
#We create variables that we will need to perform the selection of the best model and to collect the results
error = 1000000
model = NULL
list_mae = list()
list_mse = list()

for (i in 1:10){ 
  #(Step 1) Create data partitions
  training_index = createDataPartition(y = data$charges, p = 0.8, list = FALSE)

  #(Step 2) Split train and test partitions
  training_data = data[training_index,]
  test_data = data[-training_index,] 

  #(Step 3) Train the model
  model_obj <- lm(formula = charges ~. , data = training_data)

  #(Step 4) Make the predictions
  prediction <- predict(model_obj, test_data)

  #(Step 5) Perform calculations of MAE and MSE
  #Mean absolute Error = Error medio absoluto
  mae = mean(abs(prediction - test_data$charges))  
  list_mae[i] = mae
  #Mean square Error = Error cuadrático medio
  mse = mean((prediction-test_data$charges)*(prediction-test_data$charges))
  list_mse[i] = mse
  
  #(Step 6) Selection of the best model
  if (mae < error){
    error = mae
    model = model_obj
  }
}

Print the table with the iteration (1:10), with the results of MAE and MSE for each model:

In [ ]:
table = data.frame("Model_number"= 1:10, "MAE"=unlist(list_mae), "MSE"=unlist(list_mse))
knitr::kable(table)

Now we can see the Average MAE & MSE and Standar Deviation get from the 10 iterations:

In [ ]:
print(glue("Average Mean Absolute Error: {mean(table$MAE)}"))
print(glue(""))
print(glue("Average Mean Square Error: {mean(table$MSE)}"))
print(glue(""))
print(glue("Standard desviation of MAE: {sd(table$MAE)}"))
print(glue(""))
print(glue("Standard desviation of MSE: {sd(table$MSE)}"))

###Answers to the questions:

1.- On average, if a person quits smoking, how much will the charges will be reduced?

To answer to this question, first we will create a subset only with the smokers, to avoid using non-smokers in the data frame: 

In [ ]:
data = read.csv('insurance.csv')
df_smokers = data[which(data$smoker == "yes"),]

Then we are going to predict how much would the new charges be for those who now are smokers but suddently they quit smoking:

In [ ]:
df_smokers$smoker = "no"
df_smokers$new_charges = predict(model, newdata = df_smokers)

result = mean((df_smokers$charges - df_smokers$new_charges))
result = prettyNum(result, drop0trailing = 2)
#We do this to clean the result

print(glue("On average to quit smoking, for a smoking person will reduce the charge in: {result}"))

2.- Same if a non-smoker starts smoking... how much will the charges be increased?

First, we are going to create a subset of the data frame only with those who are non smokers:

In [ ]:
data = read.csv('insurance.csv')
df_nonsmokers = data[which(data$smoker == "no"),]

Now we only have to predict the new charges supposing that they start smoking and return the average:

In [ ]:
df_nonsmokers$smoker = "yes"
df_nonsmokers$new_charges = predict(model, newdata = df_nonsmokers)

result = mean((df_nonsmokers$new_charges - df_nonsmokers$charges))
result = prettyNum(result, drop0trailing = 2)
#We do this to print a nicer number

print(glue("On average to quit smoking, for a smoking person will reduce the charge in: {result}"))

3.- Which are the 3 persons whos charge will be increased most from now to 5 years?

In [ ]:
data = read.csv('insurance.csv') 
# We need to restore the original data (possible changes on smoker column for the previous exercise)

data$age <- data$age + 5 
# We travel on time

data$futureCharges <- predict(model, newdata = data) 
# We calculate the charges in the future

data$difference <-  data$futureCharges - data$charges
# We calculate a new feature, diffenrence between charges now and then


data <- data[order(-data$difference),] 
# We will reorder the dataframe, in order of descending, by the increase of insurance

result <- head(data,3) 
# We finally select the three people with mayor increasement on the following 5 yrs
print(glue("This are the three persons who's charge will be increase the most within the next five years:"))
knitr::kable(result)

## Decision Trees

### Data Description 📚

This dataset contains information about used cars. Data [can be found here](https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho) (or in the Drive).

The columns in the given dataset are as follows: name, year
selling_price, km_driven, fuel, seller_type, transmission, Owner


### You are asked to: 🚨

1. Read the provided file and store in a data.frame. Also, remove the name column. 
2. Repeat 10 times
  - Randomly divide data in train (80%) and test (20%)
    - 🔎 For this, you can use the [createDataPartition](https://www.rdocumentation.org/packages/caret/versions/6.0-86/topics/createDataPartition) function of the *caret* package as:
    - createDataPartition(y = **data$Target**, p = **percentage**, list = FALSE)
  - Train a decision tree to predict the seller_type and another one to predict the owner
    - 🔎 For this, you have to use the [rpart](https://www.rdocumentation.org/packages/rpart/versions/4.1-15/topics/rpart) function within the *caret* package as:
    - model = rpart(**Target~xxxxxxx**, **training.data**)
  - Make predictions over the test data for both decision trees
    - 🔎 You will have to use the predict function as:
    - predictions = predict(**model**,**test.data**)
  - Calculate and print both accuracies of the models over the test data.
    - The accuracy is calculated as the percertage of hit over the giving data, or the percentage of data in which the real class is equal to the predicted one.
3. Explore the *tree* objects in order to print the top-5 most relevant columns in each case. 
4. For one of the trees (at your choice) build a "Form cell" (example in next cell) that allows you to insert all the data and then, the model is executed printing the predictions.


###Your solution here 💻

Read the CSV file and create the data frame. 
(We will delete the car names in order to make it easier for the algorithm 'rpart' to run properly. We also noticed another outlier as there is only one car that its fuel is Electric, so we decided to remove that single row in order to carry out correctly the predictions).

In [ ]:
data = read.csv('CAR DETAILS FROM CAR DEKHO.csv')
data$name = NULL
data = data[- grep("Electric", data$fuel),]

Create data partitions of data, 80% training and 20% test data:

In [ ]:
seller_index = createDataPartition(y = data$seller_type, p = 0.8, list = FALSE)
owner_index = createDataPartition(y = data$owner, p = 0.8, list = FALSE)

Split training data and test data and training the model for both cases:

In [ ]:
  training_seller = data[seller_index,]
  test_seller = data[-seller_index,]

  training_owner = data[owner_index,]
  test_owner = data[-owner_index,]

  seller_model = rpart(formula = seller_type ~.,  data = training_seller)  
  owner_model = rpart(formula = owner~., data = training_owner)

  # Output
  print(glue("Size of the training set: {nrow(training_owner)}"))
  print(glue("Size of the test set: {nrow(test_owner)}"))
  print(glue("Size of the whole data set: {nrow(data)}"))

Make predictions over the test data for both decision trees:

In [ ]:
prediction_seller = predict(seller_model, test_seller, type = "class")
prediction_owner = predict(owner_model, test_owner, type = "class")

Create the confusion matrix for both models:

In [ ]:
matrix_seller = confusionMatrix(table(prediction_seller,test_seller$seller_type))
matrix_owner = confusionMatrix(table(prediction_owner,test_owner$owner))

#Print both confunsion matrix
matrix_seller$table
matrix_owner$table

Calculation of Accuracy of both models:

In [ ]:
# Print the accuracy 
print(glue("The model hits in {round(100*matrix_seller$overall[[1]],2)} % of the cars"))
print(glue("The model hits in {round(100*matrix_owner$overall[[1]],2)} % of the cars"))

###**Repeat the tests 10 times:**

Train 10 times the model and we will select the model with best results, the best according to the high accuracy.

In [ ]:
data = read.csv('CAR DETAILS FROM CAR DEKHO.csv')
data$name = NULL
data = data[- grep("Electric", data$fuel),]
#Read the CSV file and create the data frame. Also we delete the name of the cars.

model_seller = NULL
model_owner = NULL
accuracy_seller = 0
accuracy_owner = 0
# Create the variables we will use to evaluate accuracies and to store the best of the models

for (i in 1:10){
  #(Step 1) Create the data partitions
  seller_index = createDataPartition(y = data$seller_type, p = 0.8, list = FALSE)
  owner_index = createDataPartition(y = data$owner, p = 0.8, list = FALSE)

  #(Step 2) Split train and test data
  training_seller = data[seller_index,]
  test_seller = data[-seller_index,]

  training_owner = data[owner_index,]
  test_owner = data[-owner_index,]

  #(Step 3) Train the models
  seller_model = rpart(formula = seller_type ~.,  data = training_seller)
  owner_model = rpart(formula = owner~., data = training_owner)

  #(Step 4) Make the predictions for both models
  prediction_seller = predict(seller_model, test_seller, type = "class")
  prediction_owner = predict(owner_model, test_owner, type = "class")
  
  #(Step 5) Calculate the confusion matrix for both models
  matrix_seller = confusionMatrix(table(prediction_seller,test_seller$seller_type))
  matrix_owner = confusionMatrix(table(prediction_owner,test_owner$owner))

  #(Step 6) Evaluate the accuracy of the models, and if is better than the previous model we will store them in our variables
  if (accuracy_seller <= round(100*matrix_seller$overall[[1]],2)){
    model_seller = seller_model
    accuracy_seller = round(100*matrix_seller$overall[[1]],2)
  }

  if (accuracy_owner <= round(100*matrix_owner$overall[[1]],2)){
    model_owner = owner_model
    accuracy_owner = round(100*matrix_owner$overall[[1]],2)
  }
}

As a result of the repetition of the tests 10 times, we have determined that the following models have beem the best with the following accuracies:

In [ ]:
# Print the accuracy 
print(glue("The model with the target of the seller type hits in {accuracy_seller} % of the cars"))
print(glue("The model with the target of the owner type hits in {accuracy_owner} % of the cars"))


### Most valuable columns for the models

In this graphs we can see the different divisions, by the features of the data frame, and their respectives values, for both of the tree decisions. 

The models have a correct way to divide the test data in order to carry out the predictions. As we see here, there are ordered by importance of relevance for the algorithm to carry out the calculations.

In [ ]:
rpart.plot(model_owner, extra = 101)
model_owner
rpart.plot(model_seller)
model_seller

###"Form Cells"

We have created three form cells, the first two are for the models that we already had to use before, and we wanted to add a last one to predict the selling price.


In [ ]:
#@title Introduce here the values to predict owner:

year <- 2011 #@param {type:"slider", min:1989, max:2022, step:1}
km_driven <- 60660 #@param {type:"slider", min:0, max:1000000, step:1}
fuel <- "CNG" #@param ["Petrol", "Diesel", "CNG", "LPG"]
seller_type <- "Dealer" #@param ["Individual", "Dealer", "Trustmark Dealer"]
transmission <- "Automatic" #@param ["Manual", "Automatic"]
#(Don't) owner <- "First Owner" #@param ["First Owner", "Second Owner", "Third Owner", "Fourth & Above Owner","Test Drive Car"]
selling_price <- 60660 #@param {type:"slider", min:0, max:1000000, step:1}

print(glue("I can do all the operations I need here using the defined values"))

print(glue("Selected value for the year is {year}"))
print(glue("Selected value for KM driven is {km_driven}"))
print(glue("Selected value for fuel is {fuel}"))
print(glue("Selected value for seller type is {seller_type}"))
print(glue("Selected value for transmission is {transmission}"))
#print(glue("Selected value for owner is {owner}"))
print(glue("Selected value for the selling price is {selling_price}"))


The answers will appear with a percentage of confidence for each of the answers.

In [ ]:
df = data.frame(year = year, km_driven = km_driven, fuel = fuel, seller_type = seller_type,  transmission = transmission, selling_price = selling_price)
data = read.csv('CAR DETAILS FROM CAR DEKHO.csv')
data$name = NULL

#Print as percentage
prediction = predict(model_owner, df)
prediction

#Print the predicted value
prediction = predict(model_owner, df, type = 'class')
print(glue(""))
prediction



In [ ]:
#@title Introduce here the values to predict seller type:

year <- 1997 #@param {type:"slider", min:1989, max:2022, step:1}
km_driven <- 142579 #@param {type:"slider", min:0, max:1000000, step:1}
fuel <- "Petrol" #@param ["Petrol", "Diesel", "CNG", "LPG"]
#seller_type <- "Individual" #@param ["Individual", "Dealer", "Trustmark Dealer"]
transmission <- "Manual" #@param ["Manual", "Automatic"]
owner <- "First Owner" #@param ["First Owner", "Second Owner", "Third Owner", "Fourth & Above Owner","Test Drive Car"]
selling_price <- 3910 #@param {type:"slider", min:0, max:1000000, step:1}

print(glue("I can do all the operations I need here using the defined values"))

print(glue("Selected value for the year is {year}"))
print(glue("Selected value for KM driven is {km_driven}"))
print(glue("Selected value for fuel is {fuel}"))
#print(glue("Selected value for seller type is {seller_type}"))
print(glue("Selected value for transmission is {transmission}"))
print(glue("Selected value for owner is {owner}"))
print(glue("Selected value for the selling price is {selling_price}"))


The answers will appear with a percentage of confidence for each of the answers.

In [ ]:
df = data.frame(year = year, km_driven = km_driven, fuel = fuel, seller_type = seller_type,  transmission = transmission, selling_price = selling_price)
data = read.csv('CAR DETAILS FROM CAR DEKHO.csv')
data$name = NULL

#Print as percentage
prediction = predict(model_seller, df)
prediction

#Print the predicted value
prediction = predict(model_seller, df, type = 'class')
print(glue(""))
prediction

Finally, we are going to create the new model with the **selling price** as a target, because the former two Form Cells were using a model that was created already. We wanted to do this prediction of the selling price because the other two have not felt a challenge at all.

In [ ]:
data = read.csv('CAR DETAILS FROM CAR DEKHO.csv')
data$name = NULL
data = data[- grep("Electric", data$fuel),]
#Read the CSV file and create the data frame. Also we delete the column Name of the cars.

  #(Step 1) Create the data partitions
  price_index = createDataPartition(y = data$selling_price, p = 0.8, list = FALSE)

  #(Step 2) Split train and test data
  training_price = data[price_index,]
  test_price = data[-price_index,]

  #(Step 3) Train the model with the target of the selling price
  price_model = rpart(formula = selling_price ~.,  data = training_price)

 print(glue("The model of the selling price is ready :)"))

Now that we have the model trained we only need to select the values and make the predition.

In [ ]:
#@title Introduce here the values to predict selling price:

year <- 2019 #@param {type:"slider", min:1989, max:2022, step:1}
km_driven <- 0 #@param {type:"slider", min:0, max:1000000, step:1}
fuel <- "Petrol" #@param ["Petrol", "Diesel", "CNG", "LPG"]
seller_type <- "Dealer" #@param ["Individual", "Dealer", "Trustmark Dealer"]
transmission <- "Automatic" #@param ["Manual", "Automatic"]
owner <- "First Owner" #@param ["First Owner", "Second Owner", "Third Owner", "Fourth & Above Owner","Test Drive Car"]
#selling_price <- 60660 #@param {type:"slider", min:0, max:1000000, step:1}

print(glue("Selected value for the year is {year}"))
print(glue("Selected value for KM driven is {km_driven}"))
print(glue("Selected value for fuel is {fuel}"))
print(glue("Selected value for seller type is {seller_type}"))
print(glue("Selected value for transmission is {transmission}"))
print(glue("Selected value for owner is {owner}"))
#print(glue("Selected value for the selling price is {selling_price}"))


The answers will appear with a percentage of confidence for each of the answers.

In [ ]:
df = data.frame(year = year, km_driven = km_driven, fuel = fuel, seller_type = seller_type,  transmission = transmission, selling_price = selling_price)
d3ata = read.csv('CAR DETAILS FROM CAR DEKHO.csv')
data$name = NULL

#Print the selling price prediction value
prediction = predict(price_model, df)
prediction = prettyNum(prediction, drop0trailing = 2)
print(glue("The predicted selling price for a car with the values above is: {prediction}"))
